# Planning Lab 1: Modeling planning domains and problems

This lab will explore PDDL (Planning Domain Definition Language) and its application in defining planning domains and problems. As our running example we will use the Blocksword domain, one of the most common benchmarks in planning research.

#### At the end of this lab you will be able to:
- Define a PDDL domain for a planning benchmark domain,
- Create problem instances to challenge the planning process

## The Blocksworld Domain

<img src="images/blocksworld.png">

The Blocksworld planning domain is a well-known problem in automated planning. It involves a set of blocks and a table, where the task is to move the blocks from an initial arrangement to a desired goal configuration. Each block can be individually placed on the table or stacked on another block, with a limited set of actions available for picking up, putting down, stacking, and unstacking blocks. The challenge lies in determining a sequence of moves that will transform the initial configuration into the goal configuration while respecting constraints such as block order and availability. Although simple in concept, Blocksworld highlights fundamental planning challenges like ordering dependencies, managing subgoal interactions, and efficiently exploring possible sequences of actions.

**We will now model the Blocksworld domain using PDDL syntax**

## PDDL Domain definition

### Types
- **Blocks**: Each block has a unique identifier (e.g. A, B, C) and can be stacked on top of other blocks or placed on the table. The blocks are homogeneous in size, shape, and weight, so they can be freely moved or stacked.
- **Table**: The table acts as a foundational surface where blocks can be placed individually when not stacked on other blocks. We will not explicitly model the table in the domain definition, since we will use a special predicate to represent it.

### State Representation (Fluents):
- **On(Block1, Block2)**: A relation that denotes Block1 is on top of Block2.
- **OnTable(Block)**: A relation indicating that a block is directly on the table, this predicate allows us to represent the interaction with the table.
- **Clear(Block)**: A relation indicating that a block has nothing on top of it and can thus be moved.
- **Holding(Block)**: Represents the agent holding a specific block. The agent can only hold one block at a time.
- **HandEmpty**: A state indicating that the agent is not holding anything.

**Below you can see how this definition can be represented in PDDL:**

In [1]:
domain_model = '''(define (domain blocksworld)

	(:requirements :typing :fluents :negative-preconditions)

	(:types
		block
		; we do not need a table type as we use the ontable predicate
	)

	(:predicates
		(on ?a ?b - block) ; block `?a` is top of block `?b`
		(ontable ?a - block) ; block `?a` is on table
        (clear ?a - block) ; there is nothing on top of block `?a`
		(holding ?a - block) ; gripper is holding block `?a`
		(handempty) ; gripper is not holding any block
	)
    
'''

### Actions:
- **PickUp(Block)**: The agent picks up a block from the table.  
   *Preconditions*: the block is on the table and clear.  
   *Effects*: the block is now held by the agent (holding), and the agent’s hand is no longer empty.
- **PutDown(Block)**: The agent places a block on the table.  
   *Preconditions*: the agent is holding the block.  
   *Effects*: the block is now on the table, and the agent's hand becomes empty.
- **Stack(Block1, Block2)**: The agent stacks one block on top of another.  
   *Preconditions*: the agent is holding Block1, and Block2 is clear.  
   *Effects*: Block1 is now on Block2, and the agent's hand is empty.
- **Unstack(Block1, Block2)**: The agent removes Block1 from on top of Block2.  
   *Preconditions*: Block1 is on Block2 and clear.  
   *Effects*: the agent is holding Block1, and Block2 is now clear.

## Exercise 1:
**Have a look at how we defined the PickUp and Unstack actions and complete the definition adding the PutDown and Stack actions:**

In [2]:
domain_actions = '''
	(:action pickup ; this action is only for picking from table
		:parameters (?a - block)
		:precondition (and
			(ontable ?a)
			(handempty)
			(clear ?a)
		)
		:effect (and
			(holding ?a)
			(not (handempty))
			(not (clear ?a))
			(not (ontable ?a))
		)
	)
	(:action unstack ; only suitable for picking from block
		:parameters (?a ?b - block)
		:precondition (and
			(on ?a ?b)
			(handempty)
			(clear ?a)
		)
		:effect (and
			(holding ?a)
			(not (handempty))
			(not (clear ?a))
			(clear ?b)
			(not (on ?a ?b))
		)
	)

    (:action putdown ; agent places a block on the table
        :parameters (?a - block)
        :precondition (holding ?a)
        :effect (and
            (ontable ?a)
            (clear ?a)
            (handempty)
            (not (holding ?a))
        )
    )

    (:action stack ; agent stacks one block on top of another
        :parameters (?a ?b - block)
        :precondition (and
            (holding ?a)
            (clear ?b)
        )
        :effect (and
            (on ?a ?b)
            (clear ?a)
            (handempty)
            (not (holding ?a))
            (not (clear ?b))
        )
    )

)
'''
with open('blocksworld_domain.pddl', 'w') as f:
    f.write("\n".join([domain_model, domain_actions]))

To verify that the definition of the domain is correct, we will now try to execute a simple plan involving the actions you had to implement. To this aim, we make use of the **unified_planning** library, which allows us to import the PDDL domain definition above and convert it into a python object.

In [3]:
from unified_planning.io import PDDLReader, PDDLWriter
from unified_planning.shortcuts import *
from unified_planning.plans import SequentialPlan, ActionInstance

# here we import the domain and a test problem instance:
reader = PDDLReader()
problem = reader.parse_problem('blocksworld_domain.pddl', 'test_actions.pddl') 

# here we manually create a simple plan that involves the actions you defined:
a = problem.object("a")
b = problem.object("b")
c = problem.object("c")

try:
    putdown = problem.action("putdown")
    pickup = problem.action("pickup")
    stack = problem.action("stack")
    plan = SequentialPlan([
        ActionInstance(putdown, (a,)), 
        ActionInstance(pickup, (b,)),  
        ActionInstance(stack, (b, c))  
    ])
except Exception:
    print('''You have to define the putdown and stack actions before proceeding with the notebook!\n 
    If you did it, try to rerun the notebook cells from the start!''')
else:
    
    # we now check if a real planner can solve a simple problem instance, 
    # to which the previously defined plan is the solution:
    with OneshotPlanner(name="fast-downward") as planner:
        result = planner.solve(problem)
        is_valid = result.plan == plan
        if is_valid:
            print("The planner solved the test problem using the actions you defined!")
        else:
            print("The generated plan is not optimal, your action definition might be incomplete!")


NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 550 of `C:\Users\filip\OneDrive\Desktop\Lavoro\Git\Planning-lab\planning-lab-venv\Lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

The planner solved the test problem using the actions you defined!


**Now that the domain instance is complete, let's try to define a problem instance. We will make use of the PDDL language again.**

## EXERCISE 2
Let's try to represent the initial state and goal state depicted in the figure:

<img src="images/blocksworld-example.png" width="800">

**Have a look at the problem definition above and add the missing fluents to describe the situation depicted in the figure:**

In [4]:
problem_instance = '''
(define (problem demo)(:domain blocksworld)

(:objects
    A B C - block
    )

(:init
    (ontable A) ; pink block
    (on C A)(clear C) ; violet block on pink block
    
    (ontable B)
    (clear B) ; ADD FLUENTS HERE TO REPRESENT THE LIGHT-BLUE BLOCK B

    (handempty)
)

(:goal (and
    (on A B)
    (clear A)
    
    (on B C)
    
    (ontable C)
    
    (handempty) ; ADD FLUENTS TO COMPLETE THE GOAL STATE DEFINITION
    
))
)
'''

In [5]:
with open('blocksworld_problem1.pddl', 'w') as f:
    f.write(problem_instance)

problem = reader.parse_problem('blocksworld_domain.pddl', 'blocksworld_problem1.pddl')
print(problem)

problem name = demo

types = [block]

fluents = [
  bool on[a=block, b=block]
  bool ontable[a=block]
  bool clear[a=block]
  bool holding[a=block]
  bool handempty
]

actions = [
  action pickup(block a) {
    preconditions = [
      (ontable(a) and handempty and clear(a))
    ]
    effects = [
      holding(a) := true
      handempty := false
      clear(a) := false
      ontable(a) := false
    ]
  }
  action unstack(block a, block b) {
    preconditions = [
      (on(a, b) and handempty and clear(a))
    ]
    effects = [
      holding(a) := true
      handempty := false
      clear(a) := false
      clear(b) := true
      on(a, b) := false
    ]
  }
  action putdown(block a) {
    preconditions = [
      holding(a)
    ]
    effects = [
      ontable(a) := true
      clear(a) := true
      handempty := true
      holding(a) := false
    ]
  }
  action stack(block a, block b) {
    preconditions = [
      (holding(a) and clear(b))
    ]
    effects = [
      on(a, b) := true
      

**Now that our problem instance is complete, let's invoke a state-of-the-art planner (Fast Downward - https://github.com/aibasel/downward/blob/main/README.md) to see if there exists a plan (i.e. a sequence of actions) that can lead from the initial state to the goal state. If you correctly defined the initial and final state the planner should return a solution:**

In [6]:
import unified_planning
from unified_planning.shortcuts import *

with OneshotPlanner() as planner:
    result = planner.solve(problem)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print('The planner found a solution!')
        print(result.plan)
    else:
        print("No plan found.")

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 550 of `C:\Users\filip\OneDrive\Desktop\Lavoro\Git\Planning-lab\planning-lab-venv\Lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

The planner found a solution!
SequentialPlan:
    unstack(c, a)
    putdown(c)
    pickup(b)
    stack(b, c)
    pickup(a)
    stack(a, b)


## Exercise 3
Now try to define a new problem instance that can be solved using the sequence of actions represented in the figure:

<img src="images/blocksworld-example2.png" width="800">

**Complete the problem definition below and test your solution with the code in the last cell**

In [7]:
problem_instance_2 = '''
(define (problem demo2)(:domain blocksworld)

(:objects
      A B C D - block
  )

(:init
    (ontable A)
    (ontable D)
    (clear D)
    (on C B)
    (clear C)
    (on B A)

    (handempty)
    )

(:goal
    (and
    (ontable A)
    (ontable D)
    (on C A)
    (on B D)
    (handempty)
    )
    )
)


'''

In [8]:
with open('blocksworld_problem2.pddl', 'w') as f:
    f.write(problem_instance_2)
problem = reader.parse_problem('blocksworld_domain.pddl', 'blocksworld_problem2.pddl') #ADD PROBLEM INSTANCE FILE HERE
with OneshotPlanner() as planner:
    result = planner.solve(problem)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(result.plan)
    else:
        print("No plan found.") 

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 550 of `C:\Users\filip\OneDrive\Desktop\Lavoro\Git\Planning-lab\planning-lab-venv\Lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

SequentialPlan:
    unstack(c, b)
    putdown(c)
    unstack(b, a)
    stack(b, d)
    pickup(c)
    stack(c, a)


## Exercise 4
Now we want to extend our domain to work with different block sizes. Create a new domain definition, modifying the one we built in exercise 1, to represent two kinds of blocks:
- SmallBlock: a tiny block that can be put everywhere;
- BigBlock: a bigger and heavier block that can be put on the table and on another BigBlock, but not on a SmallBlock.
Complete the above specification adding new predicates to represent the weight of the blocks and define the two missing actions (putdown and stack) according to the new predicates.

In [17]:
new_domain_model = '''(define (domain sized-blocksworld)

	(:requirements :typing :fluents :negative-preconditions)

	(:types
		block
	)

	(:predicates
		(on ?a ?b - block) ; block `?a` is top of block `?b`
		(ontable ?a - block) ; block `?a` is on table
        (clear ?a - block) ; there is nothing on top of block `?a`
		(holding ?a - block) ; gripper is holding block `?a`
		(handempty) ; gripper is not holding any block


        ; YOUR CODE HERE
        ; Hint: you will need to add a 'small-block' predicate and a 'big-block' predicate
        
        (small-block ?a - block) ; block ?a is a small block
        (big-block ?a - block) ; block ?a is a big block


        
	)
    
    (:action pickup ; this action is only for picking from table
		:parameters (?a - block)
		:precondition (and
			(ontable ?a)
			(handempty)
			(clear ?a)
		)
		:effect (and
			(holding ?a)
			(not (handempty))
			(not (clear ?a))
			(not (ontable ?a))
		)
	)
	(:action unstack ; only suitable for picking from block
		:parameters (?a ?b - block)
		:precondition (and
			(on ?a ?b)
			(handempty)
			(clear ?a)
		)
		:effect (and
			(holding ?a)
			(not (handempty))
			(not (clear ?a))
			(clear ?b)
			(not (on ?a ?b))
		)
	)



    ; YOUR CODE HERE

    (:action stack
        :parameters (?a ?b - block)
        :precondition (and
            (or
                (small-block ?a)
                (and (big-block ?a) (big-block ?b))
            )
            (holding ?a)
            (clear ?b)
            )
        :effect (and
            (on ?a ?b)
            (clear ?a)
            (handempty)
            (not (holding ?a))
            (not (clear ?b))
        ))

    (:action putdown
        :parameters (?a - block)
        :precondition (holding ?a)
        :effect (and
            (ontable ?a)
            (clear ?a)
            (not (holding ?a))
            (handempty)
        ))
)
'''
with open('sized_blocksworld_domain.pddl', 'w') as f:
    f.write(new_domain_model)

We will now try solving two simple problem instances to verify that the domain definition is correct. We will ask the planner to find a plan for the two scenarios depicted in the image:

<img src="images/two_sizes_blocks.png">

The planner should be able to solve scenario 1, but it shouldn't find a plan for scenario 2.

In [18]:
try:
    problem1 = reader.parse_problem('sized_blocksworld_domain.pddl', 'sized_problem1.pddl') 
    problem2 = reader.parse_problem('sized_blocksworld_domain.pddl', 'sized_problem2.pddl') 
    with OneshotPlanner() as planner:
        # Searching for a plan to solve scenario 1
        result = planner.solve(problem1)
        if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("A plan was found for scenario 1:\n")
            print(result.plan)
        else:
            print("The planner didn't find a way to solve scenario 1, check the domain definition again!") 
        # Searching for a plan to solve scenario 2
        result = planner.solve(problem2)
        if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("A plan was found for scenario 2, this should not happen!\nCheck the obtained plan to see what's wrong:")
            print(result.plan)
        else:
            print("\n\nThe planner didn't find a way to solve the impossible scenario 2!") 
except:
    print("\nThe parser can't process the problem definition, did you define the small-block and big-block predicates correctly?")

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 550 of `C:\Users\filip\OneDrive\Desktop\Lavoro\Git\Planning-lab\planning-lab-venv\Lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

A plan was found for scenario 1:

SequentialPlan:
    unstack(c, a)
    putdown(c)
    pickup(b)
    stack(b, a)
    pickup(c)
    stack(c, b)


The planner didn't find a way to solve the impossible scenario 2!


## Question
**What happens if we model the goal state of Scenario 2 as the start state of a new problem instance?**